In [3]:
import os
import subprocess
from IPython.display import display, HTML, Image
import io
from operator import itemgetter

from TAS_Python_Utilities import data_viz
from TAS_Python_Utilities import data_viz_target
from TAS_Python_Utilities import visualize_tree

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.plotly as py

from sklearn.tree import export_graphviz
from sklearn import tree
from sklearn import metrics
from sklearn import tree
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import linear_model
from sklearn import neighbors
from sklearn import neural_network

%matplotlib inline
%qtconsole

In [4]:
dataset = pd.read_csv('fashion-mnist_train.csv')
dataset.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
dataset["label"].value_counts()

9    6000
8    6000
7    6000
6    6000
5    6000
4    6000
3    6000
2    6000
1    6000
0    6000
Name: label, dtype: int64

In [6]:
X=dataset.iloc[:,1:]

In [7]:
Y=dataset["label"]

In [8]:
dataset.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
X_train_plus_valid, X_test, y_train_plus_valid, y_test \
    = train_test_split(X, Y, random_state=0, \
                                    train_size = 0.7)

X_train, X_valid, y_train, y_valid \
    = train_test_split(X_train_plus_valid, \
                                        y_train_plus_valid, \
                                        random_state=0, \
                                        train_size = 0.5/0.7)
print(type(X_train_plus_valid))
print(type(y_train_plus_valid))
print(len(y_train))
print(len(y_valid))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>
30000
12000


In [83]:
my_tree = tree.DecisionTreeClassifier(criterion="entropy")
my_tree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [84]:
# Make a set of predictions for the training data
y_pred = my_tree.predict(X_train)

# Print performance details
accuracy = metrics.accuracy_score(y_train, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_train, y_pred))

# Print confusion matrix
print(metrics.confusion_matrix(y_train, y_pred))

# Print nicer homemade confusion matrix
print("Confusion Matrix")
pd.crosstab(y_train, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 1.0
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      3011
          1       1.00      1.00      1.00      2947
          2       1.00      1.00      1.00      3014
          3       1.00      1.00      1.00      2948
          4       1.00      1.00      1.00      3026
          5       1.00      1.00      1.00      3028
          6       1.00      1.00      1.00      2961
          7       1.00      1.00      1.00      3060
          8       1.00      1.00      1.00      3019
          9       1.00      1.00      1.00      2986

avg / total       1.00      1.00      1.00     30000

[[3011    0    0    0    0    0    0    0    0    0]
 [   0 2947    0    0    0    0    0    0    0    0]
 [   0    0 3014    0    0    0    0    0    0    0]
 [   0    0    0 2948    0    0    0    0    0    0]
 [   0    0    0    0 3026    0    0    0    0    0]
 [   0    0    0    0    0 3028    0    0    0    0]
 [   0    0    0    0    0   

Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,3011,0,0,0,0,0,0,0,0,0,3011
1,0,2947,0,0,0,0,0,0,0,0,2947
2,0,0,3014,0,0,0,0,0,0,0,3014
3,0,0,0,2948,0,0,0,0,0,0,2948
4,0,0,0,0,3026,0,0,0,0,0,3026
5,0,0,0,0,0,3028,0,0,0,0,3028
6,0,0,0,0,0,0,2961,0,0,0,2961
7,0,0,0,0,0,0,0,3060,0,0,3060
8,0,0,0,0,0,0,0,0,3019,0,3019


In [86]:
# Make a set of predictions for the test data
y_pred = my_tree.predict(X_valid)

# Print performance details
accuracy = metrics.accuracy_score(y_valid, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_valid, y_pred))

# Print confusion matrix
print(metrics.confusion_matrix(y_valid, y_pred))

# Print nicer confusion matrix
print("Confusion Matrix")
pd.crosstab(y_valid, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.79375
             precision    recall  f1-score   support

          0       0.75      0.74      0.74      1190
          1       0.94      0.95      0.94      1210
          2       0.66      0.69      0.68      1178
          3       0.80      0.80      0.80      1203
          4       0.67      0.66      0.67      1209
          5       0.89      0.90      0.89      1186
          6       0.55      0.53      0.54      1252
          7       0.86      0.86      0.86      1151
          8       0.92      0.91      0.92      1186
          9       0.91      0.90      0.90      1235

avg / total       0.79      0.79      0.79     12000

[[ 884    8   25   48   10    2  198    1   12    2]
 [   8 1144    5   42   10    0    1    0    0    0]
 [  18    5  817   22  154    1  149    0   12    0]
 [  59   35   19  968   62    1   51    0    8    0]
 [   4    3  202   60  800    3  127    0    9    1]
 [   0    1    0    1    0 1064    3   72   17   28]
 [ 200   11  151   55  14

Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,884,8,25,48,10,2,198,1,12,2,1190
1,8,1144,5,42,10,0,1,0,0,0,1210
2,18,5,817,22,154,1,149,0,12,0,1178
3,59,35,19,968,62,1,51,0,8,0,1203
4,4,3,202,60,800,3,127,0,9,1,1209
5,0,1,0,1,0,1064,3,72,17,28,1186
6,200,11,151,55,140,3,664,0,26,2,1252
7,1,0,0,0,0,73,1,989,7,80,1151
8,10,6,12,11,10,17,23,12,1083,2,1186


In [40]:
my_tree = tree.DecisionTreeClassifier(max_depth = 12)
my_tree.fit(X_train_plus_valid, y_train_plus_valid)
my_tree.classes_

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

In [41]:
# Set up the parameter grid to seaerch
param_grid ={'criterion': ['gini', "entropy"], \
             'max_depth': list(range(3, 20, 3)), \
             'min_samples_split': [50] }

# Perform the search
my_tuned_tree = GridSearchCV(tree.DecisionTreeClassifier(), \
                                param_grid, cv=2, verbose = 0, \
                            return_train_score=True)
my_tuned_tree.fit(X_train_plus_valid, y_train_plus_valid)

# Print details
print("Best parameters set found on development set:")
display(my_tuned_tree.best_params_)
display(my_tuned_tree.best_score_)
display(my_tuned_tree.cv_results_)

Best parameters set found on development set:


{'criterion': 'entropy', 'max_depth': 12, 'min_samples_split': 50}

0.79142857142857148

{'mean_fit_time': array([  1.77180982,   3.53038061,   5.44797778,   7.27032006,
          8.33513367,   9.2926954 ,   2.16725957,   4.97121108,
          9.58244312,  11.11704326,  11.97381866,  11.60333443]),
 'mean_score_time': array([ 0.05514348,  0.0566504 ,  0.0571537 ,  0.06417215,  0.05965817,
         0.06016159,  0.05514586,  0.05715263,  0.0591737 ,  0.06015944,
         0.07319999,  0.06469154]),
 'mean_test_score': array([ 0.50069048,  0.7247381 ,  0.77980952,  0.78642857,  0.78507143,
         0.7802619 ,  0.52452381,  0.73097619,  0.78585714,  0.79142857,
         0.78878571,  0.78866667]),
 'mean_train_score': array([ 0.50302449,  0.73940505,  0.8167388 ,  0.8585948 ,  0.8729048 ,
         0.877405  ,  0.52726376,  0.74997764,  0.82664473,  0.8595008 ,
         0.87221425,  0.87388083]),
 'param_criterion': masked_array(data = ['gini' 'gini' 'gini' 'gini' 'gini' 'gini' 'entropy' 'entropy' 'entropy'
  'entropy' 'entropy' 'entropy'],
              mask = [False False Fals

In [92]:
# Make a set of predictions for the test data
y_pred = my_tuned_tree.predict(X_test)

# Print performance details
accuracy = metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.813722222222
             precision    recall  f1-score   support

          0       0.77      0.76      0.77      1799
          1       0.94      0.93      0.94      1843
          2       0.72      0.74      0.73      1808
          3       0.83      0.80      0.81      1849
          4       0.67      0.75      0.71      1765
          5       0.92      0.88      0.90      1786
          6       0.60      0.55      0.57      1787
          7       0.86      0.91      0.88      1789
          8       0.92      0.91      0.91      1795
          9       0.91      0.91      0.91      1779

avg / total       0.81      0.81      0.81     18000

Confusion Matrix


Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,1372,13,32,74,11,4,265,1,27,0,1799
1,14,1720,15,62,19,0,9,0,4,0,1843
2,23,1,1341,23,274,2,132,0,12,0,1808
3,87,68,16,1477,126,4,62,0,9,0,1849
4,7,3,214,70,1326,3,132,0,10,0,1765
5,4,9,2,2,2,1564,1,130,26,46,1786
6,261,6,227,57,206,2,981,1,46,0,1787
7,0,0,0,0,0,51,0,1624,9,105,1789
8,11,1,19,20,24,23,48,15,1630,4,1795


In [10]:
# Do the same job with random forests
my_model = ensemble.RandomForestClassifier(n_estimators=300, \
                                           max_features = 3,\
                                           min_samples_split=200)
my_model.fit(X_train,y_train)
#help(ensemble.RandomForestClassifier)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=3, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=200, min_weight_fraction_leaf=0.0,
            n_estimators=300, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [11]:
# Make a set of predictions for the test data
y_pred = my_model.predict(X_valid)

# Print performance details
accuracy = metrics.accuracy_score(y_valid, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_valid, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(y_valid, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.824833333333
             precision    recall  f1-score   support

          0       0.74      0.84      0.79      1190
          1       0.99      0.94      0.96      1210
          2       0.69      0.75      0.72      1178
          3       0.77      0.91      0.83      1203
          4       0.69      0.76      0.72      1209
          5       0.95      0.91      0.93      1186
          6       0.71      0.37      0.48      1252
          7       0.89      0.90      0.89      1151
          8       0.90      0.96      0.93      1186
          9       0.91      0.94      0.93      1235

avg / total       0.82      0.82      0.82     12000

Confusion Matrix


Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,1002,3,22,103,5,0,37,0,18,0,1190
1,2,1137,18,47,5,0,0,0,1,0,1210
2,10,0,879,11,201,3,56,0,18,0,1178
3,29,4,9,1091,32,1,26,0,11,0,1203
4,1,3,134,85,915,1,60,0,10,0,1209
5,0,0,0,0,0,1075,0,75,9,27,1186
6,310,1,203,73,157,0,459,0,49,0,1252
7,0,0,0,0,0,31,0,1037,4,79,1151
8,2,3,15,8,3,6,4,1,1142,2,1186


In [12]:
# Set up the parameter grid to seaerch
param_grid = [
 {'n_estimators': list(range(100, 200, 50)), 'max_features': list(range(8, 12, 2)), 'min_samples_split': [20]}
]

# Perform the search
my_tuned_model = GridSearchCV(ensemble.RandomForestClassifier(verbose=1), param_grid, cv=5)
my_tuned_model.fit(X_train_plus_valid, y_train_plus_valid)

# Print details
print("Best parameters set found on development set:")
print(my_tuned_model.best_params_)
print(my_tuned_model.best_score_)

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   12.3s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.2s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   12.3s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   12.2s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   12.2s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   12.4s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]

Best parameters set found on development set:
{'max_features': 10, 'min_samples_split': 20, 'n_estimators': 150}
0.867904761905


[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:   32.3s finished


In [14]:
# Make a set of predictions for the test data
y_pred = my_tuned_model.predict(X_test)

# Print performance details
accuracy = metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    1.5s finished


Accuracy: 0.871333333333
             precision    recall  f1-score   support

          0       0.81      0.85      0.83      1799
          1       1.00      0.95      0.97      1843
          2       0.78      0.82      0.80      1808
          3       0.86      0.92      0.89      1849
          4       0.76      0.82      0.79      1765
          5       0.96      0.95      0.96      1786
          6       0.73      0.56      0.63      1787
          7       0.93      0.92      0.93      1789
          8       0.94      0.97      0.95      1795
          9       0.93      0.95      0.94      1779

avg / total       0.87      0.87      0.87     18000

Confusion Matrix


Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,1525,1,26,73,9,3,139,0,23,0,1799
1,2,1758,19,47,3,0,12,0,2,0,1843
2,6,0,1481,12,205,0,82,0,22,0,1808
3,45,3,10,1697,53,0,40,0,1,0,1849
4,6,1,137,90,1451,0,69,0,11,0,1765
5,0,0,0,2,0,1700,0,56,10,18,1786
6,296,3,223,52,173,2,992,0,46,0,1787
7,0,0,0,0,0,36,0,1652,3,98,1789
8,1,0,11,7,9,5,24,2,1733,3,1795


In [15]:
y_train.values.ravel()

array([6, 9, 4, ..., 5, 7, 7], dtype=int64)

In [52]:
my_nn = neural_network.MLPClassifier(hidden_layer_sizes=(180,120,80,40))


In [53]:
my_nn.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(180, 120, 80, 40), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [54]:
# Make a set of predictions for the test data
y_pred = my_nn.predict(X_valid)

# Print performance details
accuracy = metrics.accuracy_score(y_valid, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_valid, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(y_valid, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.868666666667
             precision    recall  f1-score   support

          0       0.82      0.84      0.83      1190
          1       0.98      0.98      0.98      1210
          2       0.88      0.62      0.73      1178
          3       0.86      0.87      0.87      1203
          4       0.68      0.88      0.77      1209
          5       0.96      0.97      0.96      1186
          6       0.72      0.70      0.71      1252
          7       0.95      0.92      0.93      1151
          8       0.97      0.95      0.96      1186
          9       0.94      0.96      0.95      1235

avg / total       0.87      0.87      0.87     12000

Confusion Matrix


Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,997,3,16,46,6,0,116,0,6,0,1190
1,1,1185,0,22,1,0,1,0,0,0,1210
2,11,4,729,17,295,0,115,0,7,0,1178
3,29,19,8,1051,66,0,26,0,3,1,1203
4,0,1,28,36,1064,1,75,0,4,0,1209
5,1,0,0,0,0,1145,0,21,4,15,1186
6,172,2,49,39,108,0,873,0,9,0,1252
7,0,0,0,0,0,29,0,1062,1,59,1151
8,2,1,3,9,17,7,10,5,1131,1,1186


42000

In [30]:
# Set up the parameter grid to seaerch
param_grid ={'activation': ["relu","tanh","logistic"], \
             'alpha': [0.01, 0.001], \
             #'hidden_layer_sizes' : [(180,120,80,10),(200,150,50)]}
             'hidden_layer_sizes' : [(18,12,8),(20,15,5)]}

# Perform the search
my_tuned_nn = GridSearchCV(neural_network.MLPClassifier(), \
                                param_grid, cv=2, verbose = 1, \
                            return_train_score=True)

my_tuned_nn.fit(X_train_plus_valid, y_train_plus_valid)


Fitting 2 folds for each of 12 candidates, totalling 24 fits


[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:  8.8min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'activation': ['relu', 'tanh', 'logistic'], 'alpha': [0.01, 0.001], 'hidden_layer_sizes': [(18, 12, 8), (20, 15, 5)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [31]:
# Make a set of predictions for the test data
y_pred = my_tuned_nn.predict(X_test)

# Print performance details
accuracy = metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.799388888889
             precision    recall  f1-score   support

          0       0.81      0.75      0.78      1799
          1       0.99      0.96      0.97      1843
          2       0.68      0.57      0.62      1808
          3       0.87      0.84      0.85      1849
          4       0.51      0.61      0.55      1765
          5       0.83      0.96      0.89      1786
          6       0.54      0.60      0.57      1787
          7       0.96      0.85      0.90      1789
          8       0.95      0.94      0.95      1795
          9       0.94      0.89      0.92      1779

avg / total       0.81      0.80      0.80     18000

Confusion Matrix


Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,1356,1,6,94,25,3,293,0,21,0,1799
1,1,1769,16,37,10,0,8,0,2,0,1843
2,11,3,1036,6,566,0,181,0,5,0,1808
3,58,16,8,1548,101,1,116,0,1,0,1849
4,6,0,394,47,1083,1,230,0,4,0,1765
5,0,1,0,1,0,1717,5,26,10,26,1786
6,232,3,59,39,351,2,1068,0,33,0,1787
7,0,0,0,0,0,182,0,1527,7,73,1789
8,2,0,3,4,7,6,66,9,1695,3,1795


In [32]:
#Test karo ab

In [36]:
testset = pd.read_csv('fashion-mnist_test.csv')
testset.head()
X=testset.iloc[:,1:]
Y=testset["label"]

In [38]:
X_test=X
y_test=Y

In [42]:
########## DecisionTreeClassifier#
# Make a set of predictions for the training data
y_pred = my_tree.predict(X_test)

# Print performance details
accuracy = metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print(metrics.confusion_matrix(y_test, y_pred))

# Print nicer homemade confusion matrix
print("Confusion Matrix")
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.8128
             precision    recall  f1-score   support

          0       0.75      0.77      0.76      1000
          1       0.96      0.95      0.95      1000
          2       0.72      0.69      0.70      1000
          3       0.83      0.83      0.83      1000
          4       0.69      0.76      0.72      1000
          5       0.92      0.87      0.89      1000
          6       0.59      0.55      0.57      1000
          7       0.86      0.88      0.87      1000
          8       0.92      0.91      0.92      1000
          9       0.90      0.91      0.90      1000

avg / total       0.81      0.81      0.81     10000

[[774   5  13  42  12   1 141   0  10   2]
 [ 11 950   3  24   4   3   3   1   1   0]
 [ 10   6 691  12 154   0 119   0   8   0]
 [ 42  16  17 831  51   4  32   0   7   0]
 [ 13   0 110  39 762   0  67   0   9   0]
 [  2   4   0   3   1 869   3  67  21  30]
 [167   6 108  37 117   1 548   0  13   3]
 [  0   0   0   0   0  50   1 884   1  64]


Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,774,5,13,42,12,1,141,0,10,2,1000
1,11,950,3,24,4,3,3,1,1,0,1000
2,10,6,691,12,154,0,119,0,8,0,1000
3,42,16,17,831,51,4,32,0,7,0,1000
4,13,0,110,39,762,0,67,0,9,0,1000
5,2,4,0,3,1,869,3,67,21,30,1000
6,167,6,108,37,117,1,548,0,13,3,1000
7,0,0,0,0,0,50,1,884,1,64,1000
8,6,4,18,7,11,9,20,8,914,3,1000


In [43]:
#### Tuned DecisionTreeClassifier ##
# Make a set of predictions for the test data
y_pred = my_tuned_tree.predict(X_test)

# Print performance details
accuracy = metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.8129
             precision    recall  f1-score   support

          0       0.76      0.76      0.76      1000
          1       0.95      0.94      0.94      1000
          2       0.73      0.71      0.72      1000
          3       0.86      0.81      0.83      1000
          4       0.70      0.79      0.74      1000
          5       0.91      0.86      0.88      1000
          6       0.59      0.56      0.57      1000
          7       0.84      0.90      0.87      1000
          8       0.90      0.92      0.91      1000
          9       0.91      0.89      0.90      1000

avg / total       0.81      0.81      0.81     10000

Confusion Matrix


Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,760,4,15,39,3,3,150,0,26,0,1000
1,10,941,3,29,6,0,7,0,4,0,1000
2,13,1,708,10,160,1,97,0,10,0,1000
3,42,33,12,809,57,1,41,0,5,0,1000
4,4,1,100,24,788,0,80,0,3,0,1000
5,6,4,3,0,0,859,0,85,22,21,1000
6,152,7,121,27,108,0,562,0,23,0,1000
7,0,0,0,0,0,39,0,895,4,62,1000
8,11,1,10,4,7,17,20,10,919,1,1000


In [44]:
####### ensemble.RandomForestClassifier ###
# Make a set of predictions for the test data
y_pred = my_model.predict(X_test)

# Print performance details
accuracy = metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.8245
             precision    recall  f1-score   support

          0       0.73      0.82      0.77      1000
          1       0.98      0.93      0.96      1000
          2       0.71      0.76      0.73      1000
          3       0.76      0.91      0.83      1000
          4       0.70      0.77      0.73      1000
          5       0.95      0.90      0.93      1000
          6       0.75      0.38      0.51      1000
          7       0.88      0.87      0.87      1000
          8       0.91      0.97      0.94      1000
          9       0.88      0.93      0.91      1000

avg / total       0.83      0.82      0.82     10000

Confusion Matrix


Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,816,0,26,101,5,0,25,0,27,0,1000
1,2,934,22,35,4,1,1,0,1,0,1000
2,8,1,759,10,163,1,41,0,17,0,1000
3,27,7,11,912,26,0,14,0,3,0,1000
4,0,2,108,72,767,1,42,0,8,0,1000
5,0,0,0,0,0,904,0,63,7,26,1000
6,264,4,136,60,121,0,382,0,33,0,1000
7,0,0,0,0,0,29,0,874,0,97,1000
8,1,1,13,3,5,2,1,5,968,1,1000


In [45]:
#######Tuned ensemble.RandomForestClassifier ###
# Make a set of predictions for the test data
y_pred = my_tuned_model.predict(X_test)

# Print performance details
accuracy = metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.8711
             precision    recall  f1-score   support

          0       0.79      0.84      0.81      1000
          1       0.99      0.96      0.97      1000
          2       0.79      0.79      0.79      1000
          3       0.88      0.93      0.90      1000
          4       0.79      0.85      0.82      1000
          5       0.97      0.94      0.95      1000
          6       0.72      0.57      0.64      1000
          7       0.91      0.91      0.91      1000
          8       0.94      0.97      0.95      1000
          9       0.92      0.94      0.93      1000

avg / total       0.87      0.87      0.87     10000

Confusion Matrix


[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed:    0.7s finished


Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,841,0,14,40,1,1,86,0,17,0,1000
1,3,961,10,17,2,1,6,0,0,0,1000
2,7,1,790,9,123,0,60,0,10,0,1000
3,20,7,7,928,20,0,18,0,0,0,1000
4,0,1,65,37,855,0,38,0,4,0,1000
5,0,0,0,0,0,940,0,43,5,12,1000
6,193,1,105,26,83,0,571,0,21,0,1000
7,0,0,0,0,0,21,0,912,0,67,1000
8,1,1,6,0,5,1,12,4,968,2,1000


In [55]:
######## neural_network.MLPClassifier model ##
# Make a set of predictions for the test data
y_pred = my_nn.predict(X_test)

# Print performance details
accuracy = metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.8625
             precision    recall  f1-score   support

          0       0.80      0.82      0.81      1000
          1       0.97      0.98      0.98      1000
          2       0.89      0.61      0.73      1000
          3       0.88      0.89      0.88      1000
          4       0.70      0.90      0.79      1000
          5       0.96      0.94      0.95      1000
          6       0.66      0.66      0.66      1000
          7       0.93      0.91      0.92      1000
          8       0.98      0.95      0.96      1000
          9       0.92      0.96      0.94      1000

avg / total       0.87      0.86      0.86     10000

Confusion Matrix


Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,816,1,15,32,5,0,124,0,7,0,1000
1,5,983,0,11,0,0,1,0,0,0,1000
2,12,2,614,17,228,0,125,0,2,0,1000
3,24,21,4,892,38,0,19,0,2,0,1000
4,1,1,14,21,903,0,58,0,2,0,1000
5,0,1,1,0,0,941,0,36,4,17,1000
6,154,6,35,42,102,0,655,0,5,1,1000
7,0,0,0,0,0,26,0,912,0,62,1000
8,5,0,5,2,19,4,9,1,952,3,1000


In [47]:
######## Tuned neural_network.MLPClassifier model 
# Make a set of predictions for the test data
y_pred = my_tuned_nn.predict(X_test)

# Print performance details
accuracy = metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.8046
             precision    recall  f1-score   support

          0       0.81      0.75      0.78      1000
          1       0.98      0.97      0.97      1000
          2       0.71      0.58      0.64      1000
          3       0.87      0.83      0.85      1000
          4       0.55      0.67      0.60      1000
          5       0.84      0.95      0.89      1000
          6       0.55      0.60      0.58      1000
          7       0.94      0.84      0.89      1000
          8       0.96      0.95      0.95      1000
          9       0.91      0.90      0.91      1000

avg / total       0.81      0.80      0.81     10000

Confusion Matrix


Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,751,1,3,60,8,0,166,0,11,0,1000
1,1,972,5,12,3,1,5,0,1,0,1000
2,5,2,579,7,290,0,116,0,1,0,1000
3,33,17,6,829,51,1,62,0,1,0,1000
4,1,0,190,20,666,1,118,0,4,0,1000
5,0,0,0,0,0,951,2,23,6,18,1000
6,138,2,27,25,189,1,605,0,13,0,1000
7,0,0,0,0,0,93,0,841,1,65,1000
8,1,1,0,1,6,5,29,8,948,1,1000
